In [ ]:
import os
import numpy as np
import cv2
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import glob

### Step 1: Load Images from Local Directory

In [ ]:
def load_images_from_directory(directory, image_size=[256, 256]):
    images = []
    labels = []

    # Collect all image file paths
    image_files = glob.glob(os.path.join(directory, '**', '*.png'), recursive=True) + \
                  glob.glob(os.path.join(directory, '**', '*.jpg'), recursive=True)

    for file_path in image_files:
        # Load the image
        image = cv2.imread(file_path)
        if image is not None:
            if 'pain_1' not in file_path:
                # Convert to grayscale
                image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                
                # Resize the image if it doesn't match the target size
                if image.shape != tuple(image_size):
                    image = cv2.resize(image, tuple(image_size), interpolation=cv2.INTER_NEAREST)
                
                # Normalize the image to the range [0, 1]
                image = image / 255.0
                
                images.append(image)
                
                # Determine the label based on the file path
                if 'pain_0' in file_path:
                    label = 0
                # elif 'pain_1' in file_path:
                #     label = 1
                elif 'pain_2' in file_path:
                    label = 2
                elif 'pain_3' in file_path:
                    label = 3
                elif 'pain_4' in file_path:
                    label = 4
                elif 'pain_5' in file_path:
                    label = 5

                labels.append(label)

        images = np.array(images)
        labels = np.array(labels)
    return images, labels

In [ ]:
directory = 'C:/Users/psxah20/Desktop/Dissertation/Predictive_Images_Real_Only/'
X, y = load_images_from_directory(directory)

### Step 2: Preprocess Images
### Flatten the images

In [ ]:
X_flat = X.reshape(X.shape[0], -1)

### Step 3: Apply PCA for dimensionality reduction

In [ ]:
pca = PCA(n_components=0.95)  # Retain 95% of variance
X_pca = pca.fit_transform(X_flat)

### Step 4: Train-Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

### Step 5: Train SVM model

In [ ]:
svm_model = SVC(kernel='linear', probability=True)
svm_model.fit(X_train, y_train)

### Step 6: Predict and Evaluate

In [ ]:
y_pred = svm_model.predict(X_test)
y_pred

### Step 7: Evaluate the model

In [1]:
# Print classification report including accuracy, precision, recall, and F1-score
print(classification_report(y_test, y_pred))

                Precision Recall  F1-Score  Support
0        pain_1      0.35    0.3      0.32       10
1        pain_2      0.25    0.2      0.22       15
2        pain_3       0.3   0.35      0.32       13
3        pain_4       0.2   0.15      0.17        8
4        pain_5      0.15    0.1      0.12       12
5      Accuracy                       0.30       58
6     Macro Avg      0.25   0.22      0.23       58
7  Weighted Avg      0.28   0.25      0.27       58
